# ARIMA

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import softmax
import os
import pickle

# File path and naming convention
directory = '/Users/zxgan/FYP_Kubernetes/Dataset/'
file_prefix = 'node_node_'
file_suffix = '_dataset.csv'
num_files = 50
output_directory = '/Users/zxgan/FYP_Kubernetes/ARIMA_Forecast_Results/'
model_directory = '/Users/zxgan/FYP_Kubernetes/ARIMA_Models/'

# Ensure output directories exist
os.makedirs(output_directory, exist_ok=True)
os.makedirs(model_directory, exist_ok=True)

# Define pod statuses and forecasting parameters
statuses = ["Running", "Pending", "Succeeded", "Failed", "Unknown"]
forecast_steps = 15  # Number of steps to forecast

# Initialize a summary list for saving results
results_summary = []

# Create a better visualization for the table
def create_better_visualization(probability_df, node_id, output_path):
    plt.figure(figsize=(12, 8))
    sns.heatmap(
        probability_df, 
        annot=True, 
        fmt=".7f", 
        cmap="coolwarm", 
        cbar_kws={'label': 'Probability'},
        linewidths=0.5,
        linecolor='white'
    )
    plt.title(f"Forecasted Probabilities for Pod Statuses - Node {node_id}", fontsize=14)
    plt.xlabel("Pod Status", fontsize=12)
    plt.ylabel("Time (HH:MM)", fontsize=12)
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    # Save the plot as an image file
    plt.tight_layout()
    plt.savefig(output_path, dpi=300)
    plt.close()
    print(f"Enhanced probability visualization saved to: {output_path}")

# Loop through all datasets
for i in range(num_files):
    file_path = f"{directory}{file_prefix}{i}{file_suffix}"
    print(f"Processing dataset: {file_path}")
    
    # Load dataset
    data = pd.read_csv(file_path)
    
    # Preprocessing: Convert the 'timestamp' column to datetime format and set as index
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.set_index('timestamp')

    # Feature Engineering
    lag_features = [1, 3, 5]
    for lag in lag_features:
        for status in statuses:
            data[f'{status}_lag_{lag}'] = data[f'pod_status_{status}'].shift(lag)

    # Create rolling averages
    for window in [3, 5]:
        for status in statuses:
            data[f'{status}_rolling_{window}'] = data[f'pod_status_{status}'].rolling(window).mean()
    
    data = data.dropna()  # Drop rows with NaN values after creating lag/rolling features
    
    probability_results = {}  # To store probabilities for this dataset

    # Loop through each pod status for forecasting
    for status in statuses:
        print(f"Processing status: {status}")
        
        # Resample and preprocess pod status data
        pod_status_series = data[f'pod_status_{status}'].resample('1T').mean().fillna(method='ffill').fillna(method='bfill')
        
        # Define ARIMA parameter grid
        p = range(0, 3) 
        d = range(0, 3) 
        q = range(0, 3)  # Adjust ranges as needed for faster or broader grid search
        pdq = list(itertools.product(p, d, q))

        # Perform grid search to find the best ARIMA parameters
        best_aic = np.inf
        best_pdq = None
        best_model = None

        for param in pdq:
            try:
                model = sm.tsa.ARIMA(pod_status_series, order=param)
                results = model.fit()
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_pdq = param
                    best_model = results
            except:
                continue

        print(f"Best ARIMA parameters for {status}: {best_pdq} with AIC: {best_aic}")
        
        # Export the trained ARIMA model
        model_filename = f"{model_directory}Node_{i}_Status_{status}_ARIMA_Model.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(best_model, f)
        print(f"ARIMA model for Node {i}, Status {status} saved to: {model_filename}")
        
        # Forecast the next 'forecast_steps'
        pod_status_forecast = best_model.forecast(steps=forecast_steps)
        
        # Apply softmax to normalize probabilities
        probabilities = softmax(pod_status_forecast.values.reshape(-1, 1), axis=0).flatten()
        probability_results[status] = probabilities

        # Collect results for the summary
        results_summary.append({
            "Node": f"Node_{i}",
            "Pod Status": status,
            "Best PDQ": best_pdq,
            "AIC": best_aic
        })

    # Create a DataFrame with probabilities for all statuses
    probability_df = pd.DataFrame(
        probability_results,
        index=pd.date_range(start=data.index[-1], periods=forecast_steps, freq='1T')
    )

    # Generate a heatmap for the forecasted probabilities
    output_heatmap_path = f"{output_directory}{file_prefix}{i}_forecast_probabilities_heatmap.png"
    create_better_visualization(probability_df, i, output_heatmap_path)

# Save the summary to a CSV file
summary_df = pd.DataFrame(results_summary)
summary_csv_path = os.path.join(output_directory, "arima_summary_results.csv")
summary_df.to_csv(summary_csv_path, index=False)
print(f"Summary results saved to: {summary_csv_path}")

# Display the summary DataFrame
print("\nARIMA Summary Results:")
print(summary_df)

Processing dataset: /Users/zxgan/FYP_Kubernetes/Dataset/node_node_0_dataset.csv
Processing status: Running


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Chec

Best ARIMA parameters for Running: (1, 0, 2) with AIC: 148.04912551111133
ARIMA model for Node 0, Status Running saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Models/Node_0_Status_Running_ARIMA_Model.pkl
Processing status: Pending


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found

Best ARIMA parameters for Pending: (0, 0, 2) with AIC: 144.52924359460127
ARIMA model for Node 0, Status Pending saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Models/Node_0_Status_Pending_ARIMA_Model.pkl
Processing status: Succeeded


...

Best ARIMA parameters for Succeeded: (2, 0, 1) with AIC: 140.39947285196084
ARIMA model for Node 49, Status Succeeded saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Models/Node_49_Status_Succeeded_ARIMA_Model.pkl
Processing status: Failed


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA paramet

Best ARIMA parameters for Failed: (1, 0, 0) with AIC: 166.87016503021934
ARIMA model for Node 49, Status Failed saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Models/Node_49_Status_Failed_ARIMA_Model.pkl
Processing status: Unknown


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Usi

Best ARIMA parameters for Unknown: (1, 0, 0) with AIC: 189.95635895150383
ARIMA model for Node 49, Status Unknown saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Models/Node_49_Status_Unknown_ARIMA_Model.pkl
Enhanced probability visualization saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Forecast_Results/node_node_49_forecast_probabilities_heatmap.png
Summary results saved to: /Users/zxgan/FYP_Kubernetes/ARIMA_Forecast_Results/arima_summary_results.csv

ARIMA Summary Results:
        Node Pod Status   Best PDQ         AIC
0     Node_0    Running  (1, 0, 2)  148.049126
1     Node_0    Pending  (0, 0, 2)  144.529244
2     Node_0  Succeeded  (2, 0, 1)  153.884009
3     Node_0     Failed  (1, 0, 1)  168.532146
4     Node_0    Unknown  (1, 0, 0)  150.720253
..       ...        ...        ...         ...
245  Node_49    Running  (1, 0, 0)  132.142191
246  Node_49    Pending  (1, 0, 0)  222.390695
247  Node_49  Succeeded  (2, 0, 1)  140.399473
248  Node_49     Failed  (1, 0, 0)  166.870165
249  Nod